# Diversity Evolutionary Algorithm

In [1]:
from ariel.body_phenotypes.robogen_lite.decoders.hi_prob_decoding import HighProbabilityDecoder
from ariel.ec.a004 import Individual, EAStep, parent_selection, EA, EAStep, AbstractEA, EASettings, Population
import numpy as np
from ariel.ec.genotypes.nde.nde import NeuralDevelopmentalEncoding
from ariel_experiments.characterize.canonical.core.toolkit import CanonicalToolKit as ctk
from ariel_experiments.characterize.population import get_full_analyzed_population

from functools import partial

from ariel_experiments.characterize.individual import analyze_neighbourhood
from ariel_experiments.characterize.population import matrix_derive_neighbourhood

HPD = HighProbabilityDecoder(20)
SEED = 42
RNG = np.random.default_rng(SEED)
nde = NeuralDevelopmentalEncoding(number_of_modules=20)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def float_creep(
        individual: list[list[float]] | list[list[list[float]]], 
        mutation_probability: float,
    ) -> list[list[float]]:
        # Prep
        ind_arr = np.array(individual)
        shape = ind_arr.shape


        # Generate mutation values
        mutator = RNG.uniform(size=shape
        )

        # Include negative mutations
        sub_mask = RNG.choice(
            [-1, 1],
            size=shape,
        )

        # Determine which positions to mutate
        do_mask = RNG.choice(
            [1, 0],
            size=shape,
            p=[mutation_probability, 1 - mutation_probability],
        )
        mutation_mask = mutator * sub_mask * do_mask
        new_genotype = ind_arr + mutation_mask
        return new_genotype.tolist()

# Settings

In [3]:
# EA settings
config = EASettings()
config.is_maximisation = False

# evaluation settings
similarity_config = ctk.create_similarity_config(
    radius_strategy=ctk.RadiusStrategy.NODE_LOCAL,
    weighting_mode=ctk.WeightingMode.LINEAR,
    max_tree_radius=None,
    missing_data_mode=ctk.MissingDataMode.SKIP_RADIUS,
    tanimoto_mode=ctk.TanimotoMode.COUNTS,
    softmax_beta=1,
    power_mean_p=1
)




# EA functions

In [4]:

def make_random_robot(genotype_size:int=64) -> Individual:
    """
    Produces an robot with only its genotype

    genotype_size is standard 64
    """
    ind = Individual()
    ind.genotype =[RNG.random(genotype_size).tolist(),RNG.random(genotype_size).tolist(),RNG.random(genotype_size).tolist()]
    ind = evaluation_diversity([ind])[0]

    return ind

def crossover(population:Population) -> Population:
    """
    Does uniform crossover
    """
    mask = RNG.random(size=np.array(population[0].genotype).shape) < 0.5 
    children = []
    for parent in population:
        child = Individual()
        child.genotype = np.where(mask, np.array(parent.genotype), np.array(parent.genotype)).tolist()
        children.append(child)
    population.extend(children)
    return population

def mutation(population: Population) -> Population:
    """
    Randomly changes genotype values by a random amount
    """
    mutation_rate = 0.01 # mutation rate to be changed
    for ind in population:
        if ind.tags.get("mut", False):
            genes = ind.genotype
            mutated =[float_creep(
                individual=genes[0],
                mutation_probability=mutation_rate, 
            ),float_creep(
                individual=genes[1],
                mutation_probability=mutation_rate, 
            ),float_creep(
                individual=genes[2],
                mutation_probability=mutation_rate, 
            )]
            ind.genotype = mutated
            ind.requires_eval = True
    return population

def evaluation_diversity(population:Population) -> Population:
    """
    Produces an individual that is evaluated
    """
    graph_population = []

    # for initial population creation
    if len(population) <= 1:
        population[0].fitness = 0.0
        population[0].requires_eval = True
        return population
    
    for ind in population:
        matrixes = nde.forward(np.array(ind.genotype))
        ind_graph = HPD.probability_matrices_to_graph(matrixes[0],matrixes[1],matrixes[2])
        graph_population.append(ind_graph)
        
    analyzed_population = get_full_analyzed_population(graph_population,analyzers=[partial(analyze_neighbourhood, config=similarity_config)], derivers=[], n_jobs=-1)
    matrix_result = matrix_derive_neighbourhood(analyzed_population.raw, config=similarity_config, symmetric=True, n_jobs=-1)
    matrix = matrix_result["similarity_matrix"]["full"]
    
    for index, ind in enumerate(population):
        if ind.requires_eval:

            ind.requires_eval = False # diversity gets evaluated each time

            fitness = sum(matrix[index]) 
            population[index].fitness = float(fitness) - 1

    return population

def survivor_selection(population: Population) -> Population:
    RNG.shuffle(population)
    current_pop_size = len(population)
    for idx in range(len(population)):
        ind_i = population[idx]
        ind_j = population[idx + 1]

        # Kill worse individual
        if ind_i.fitness > ind_j.fitness and config.is_maximisation:
            ind_j.alive = False
            ind_i.requires_eval = True # we want to re-evaluate each generation
        else:
            ind_i.alive = False
            ind_j.requires_eval = True # we want to re-evaluate each generation

        # Termination condition
        current_pop_size -= 1
        if current_pop_size <= config.target_population_size:
            break
    return population



# run


In [5]:

ops = [
        EAStep("parent_selection", parent_selection),
        EAStep("crossover", crossover),
        EAStep("mutation", mutation),
        EAStep("evaluation", evaluation_diversity),
        EAStep("survivor_selection", survivor_selection),
    ]

# population = [ make_random_robot(genotype_size) for _ in range(10)] # generating genotypes

# robot = nde.forward(np.array(population[0].genotype))
# evaluation_diversity(population=population)

In [6]:
# hyperparameters 
population_size = 10
num_of_generations = 2
genotype_size = 64 # 64 is standard in the NDE


# population
population_list = [ make_random_robot(genotype_size) for _ in range(population_size)] # generating genotypes

In [ ]:
ea = EA(
                population_list,
                operations=ops,
                num_of_generations=num_of_generations,
            )
    
ea.run()